In [1]:
import sys
import os
from dotenv import load_dotenv
load_dotenv()
os.chdir(os.getenv("PROJECT_DIRECTORY"))
sys.path.append(os.getenv("PROJECT_DIRECTORY"))

from langchain.prompts import PromptTemplate
from openai import OpenAI
import libs.openai as openaiAnalytics
import json

In [2]:
def getBrandDescription(client, brandName: str, brandWebsite: str, brandCountry: str = "world") -> str:
    """
    Retrieves the company description using OpenAI's responses API and the brandDescription prompt template.
    """
    with open("prompts/brandDescription.txt", "r", encoding="utf-8") as file:
        promptTemplate = file.read()

    prompt = PromptTemplate(
        input_variables=["companyName", "companyWebsite", "companyCountry"],
        template=promptTemplate
    ).format(
        companyName=brandName,
        companyWebsite=brandWebsite,
        companyCountry=brandCountry
    )
    response = client.responses.create(
        model="gpt-4o-mini-2024-07-18",
        tools=[{"type": "web_search_preview"}],
        input=prompt,
    )
    messagesAnnotations, messagesTexts = openaiAnalytics.getResponseInfo(response)

    return next(iter(messagesTexts.values()), "")


def getBrandIndustry(client, brandName: str, brandWebsite: str, brandDescription: str, brandCountry: str = "world") -> str:
    """
    Retrieves the company industry using OpenAI's responses API and the brandIndustry prompt template.
    """
    with open("prompts/brandIndustry.txt", "r", encoding="utf-8") as file:
        promptTemplate = file.read()

    prompt = PromptTemplate(
        input_variables=["companyName", "companyWebsite", "companyCountry", "companyDescription"],
        template=promptTemplate
    ).format(
        companyName=brandName,
        companyWebsite=brandWebsite,
        companyCountry=brandCountry,
        companyDescription=brandDescription
    )
    response = client.responses.create(
        model="gpt-4o-mini-2024-07-18",
        tools=[{"type": "web_search_preview"}],
        input=prompt,
    )
    messagesAnnotations, messagesTexts = openaiAnalytics.getResponseInfo(response)

    return next(iter(messagesTexts.values()), "")


def getBrandCompetitors(client, brandName: str, brandWebsite: str, brandDescription: str, brandIndustry: str, brandCountry: str = "world") -> dict:
    """
    Retrieves the company industry using OpenAI's responses API and the brandIndustry prompt template.
    """
    with open("prompts/brandCompetitors.txt", "r", encoding="utf-8") as file:
        promptTemplate = file.read()

    prompt = PromptTemplate(
        input_variables=["companyName", "companyWebsite", "companyCountry", "companyDescription", "companyIndustry"],
        template=promptTemplate
    ).format(
        companyName=brandName,
        companyWebsite=brandWebsite,
        companyCountry=brandCountry,
        companyDescription=brandDescription,
        companyIndustry = brandIndustry
    )
    response = client.responses.create(
        model="gpt-4o-mini-2024-07-18",
        tools=[{"type": "web_search_preview"}],
        input=prompt,
    )
    messagesAnnotations, messagesTexts = openaiAnalytics.getResponseInfo(response)
    rawJson = next(iter(messagesTexts.values()), "")

    if rawJson.startswith("```json"):
        rawJson = rawJson[len("```json"):].strip()
    
    if rawJson.endswith("```"):
        rawJson = rawJson[:-3].strip()

    return json.loads(rawJson)


def getCompanyInfo(brandName: str, brandWebsite: str, brandCountry: str = "world") -> dict:
    """
    Retrieves the company description and industry using OpenAI's responses API and prompt templates.

    Args:
        brandName (str): The name of the brand/company.
        brandWebsite (str): The website of the brand/company.
        brandCountry (str, optional): The country of the brand/company. Defaults to "world".

    Returns:
        dict: A dictionary with keys 'description' and 'industry'.
    """
    client = OpenAI()
    brandDescription = getBrandDescription(client, brandName, brandWebsite, brandCountry)
    brandIndustry = getBrandIndustry(client, brandName, brandWebsite, brandCountry, brandDescription)
    brandCompetitors = getBrandCompetitors(client, brandName, brandWebsite, brandCountry, brandDescription, brandIndustry)
    
    return {
        "description": brandDescription,
        "industry": brandIndustry,
        "competitors": brandCompetitors
    }

In [3]:
brandName = "jethr"
brandWebsite = "jethr.com"
brandCountry = "italy"
brnadInformation = getCompanyInfo(brandName, brandWebsite)
brandDescription = brnadInformation['description']
brandIndustry = brnadInformation['industry']
brandCompetitors = brnadInformation['competitors']